In [1]:
import requests as req
from bs4 import BeautifulSoup
import numpy as np
import codecs
import re
import numpy as np
import pandas as pd
import time
import igraph as ig

In [2]:
import sqlite3
conn = sqlite3.connect('test.db')

利用以下10个词作为种子节点进行爬取

In [6]:
urlroot='http://baike.baidu.com/item/'
SEED=['大数据','数据挖掘','云计算','机器学习','深度学习','自然语言处理','Hadoop','人工智能','数据库','信息检索']
l=0
r=10
g=ig.Graph()

In [7]:
g.add_vertices(len(SEED))
g.vs['name']=SEED
g.vs['dist']=0
total=1000

总共爬取1000个页面，将会得到这1000个页面当中的所有正文中的词（通过分析，我发现仅爬取正文中的内容，才能保证相关性较高）

In [8]:
while l<total:
    try:
        s=req.Session()
        url=urlroot+SEED[l]
        print('currently',l,SEED[l])
        react=s.get(url)
        react.encoding='utf-8'
        soup=BeautifulSoup(react.text)
        hreflist=soup.find_all('a',href=re.compile("item"),target='_blank')
        for x in hreflist:
            x_parent_class='stranger'
            try:
                x_parent_class=x.parent['class'][0]
            except:
                pass
            if  x_parent_class not in ['basicInfo-item name','para']: 
                continue   #做出如此限制是因为我只考虑介绍文本中的链接，避免点进其他无关链接
                
            name8=x.text.strip()
            if len(g.vs.select(name_eq=name8))>0:
                if SEED[l]!=name8:
                    g.add_edges([(SEED[l],name8)])
            else:
                SEED.append(name8)
                g.add_vertices(1)
                g.add_edges([(l,r)])
                g.vs[r]['name']=name8
                g.vs[r]['dist']=g.vs[l]['dist']+1
                r=r+1
    except Exception as wrongthing:
        print(l,wrongthing)
        SEED.append(SEED[l])
        time.sleep(0.2)
    l=l+1       
    
    
    

currently 0 大数据


C:\Users\哲超\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


currently 1 数据挖掘
currently 2 云计算
currently 3 机器学习
currently 4 深度学习
currently 5 自然语言处理
currently 6 Hadoop
currently 7 人工智能
currently 8 数据库
currently 9 信息检索
currently 10 维克托·迈尔-舍恩伯格
currently 11 大数据时代
currently 12 抽样调查
currently 13 Gartner
currently 14 决策力
currently 15 洞察
currently 16 资产
currently 17 麦肯锡
currently 18 存储
currently 19 数据
currently 20 提高
currently 21 加工
currently 22 增值
currently 23 硬币
currently 24 计算机
currently 25 分布式架构
currently 26 分布式处理
currently 27 分布式数据库
currently 28 云存储
currently 29 虚拟化技术
currently 30 云时代
currently 31 Big data
currently 32 非结构化数据
currently 33 半结构化数据
currently 34 并行处理
currently 35 分布式文件系统
currently 36 云计算平台
currently 37 存储系统
currently 38 bit
currently 39 Byte
currently 40 KB
currently 41 MB
currently 42 GB
currently 43 TB
currently 44 PB
currently 45 EB
currently 46 ZB
currently 47 YB
currently 48 BB
currently 49 NB
currently 50 DB
currently 51 Bit
currently 52 比特
currently 53 Bits
currently 54 字节
currently 55 潜在
currently 56 互联网
currently 57 表象
current

检查一些结果

In [12]:
[x for x in g.vs.select(name='深度学习')]

[igraph.Vertex(<igraph.Graph object at 0x00000203530F0C78>, 4, {'dist': 0, 'name': '深度学习'})]

In [10]:
vinfo=pd.DataFrame(np.array([range(len(g.vs)),g.vs['name'],g.vs['dist']]).T,columns=['id','name','dist'])

将爬取到的节点存进数据库，方便日后取用

In [13]:
vinfo.to_sql(name='baike'+str(total),con=conn,if_exists='replace')

In [14]:
len(SEED)

27155

将爬取到的边的信息存入数据库，日后可以进行语义网络分析

In [15]:
einfo=pd.DataFrame([tuple(x) for x in g.get_edgelist()],columns=["left","right"])

In [16]:
einfo.to_sql(name='edge'+'baike'+str(total),con=conn,if_exists='replace')